In [28]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [29]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

X = data.drop('income', axis=1)
y = data['income']

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

y = LabelEncoder().fit_transform(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [32]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [33]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [34]:
class EnhancedFCN(nn.Module):
    def __init__(self, input_size=14, hidden_size=32):
        super(EnhancedFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 2)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc1(self.act(self.fc0(x)))
        return x

In [35]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [40]:
hyperparams = {
    "num_epochs": 64,
    "batch_size": 512,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.7}, # 0.0 -> all edges, 1.0 -> no edges 
    "choose_thresholds_del": {"fc0": 0.1}, # 1.0 -> all edges, 0.0 -> no edges
    "threshold": 0.01,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 1e-4,
    "delete_after": 2,    
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 900 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, batch_size: 512, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.7}, choose_thresholds_del: {'fc0': 0.1}, threshold: 0.01, min_delta_epoch_replace: 8, window_size: 5, lr: 0.0001, delete_after: 2, task_type: classification, fully_connected: False, max_to_replace: 900"

In [41]:
dense_model = EnhancedFCN()
sparse_model = convert_dense_to_sparse_network(dense_model, layers=[dense_model.fc0, dense_model.fc1])
wandb.finish()
wandb.init(
    project="self-expanding-nets-titanic",
    name=f"titanic, {name}",
    tags=["complex model", "titanic", "multiclass", hyperparams["metric"].__class__.__name__],
)

acc amount,▇████████▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁
len_choose,▅▁█
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
n_params over train_time,▁▂▁▂▁▂▂▂▁▆▅▄▄▄▄▂▄▄▇▆▅▅▆▆▅▅▃█▇▅▅
params amount,▁▁▁▁▁▁▁▁▁▄▄▃▃▃▃▃▃▃▆▆▅▅▅▅▅▅▅████
params ratio,▇▇▇▇▅▃▃▁▃▇▇█▅▆▆▇▆▆▆▆██▇▇▇▇█▇▇██
params to replace amount,▁▁▁▁▃▅▄▆▅▃▂▁▇▅▄▂▅▅▆█▂▂▄▃▄▃▂▆▆▂▁
train loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train time,▃▁▂▁▂▁▁▁▂▂▂▃▂▂▂▆▃▃▃▃▄▄▃▃▃▄▇▄▅▇█
train_time over n_params,█▅▆▆▆▅▅▅▇▂▂▃▂▂▂▄▃▃▁▁▂▂▂▂▂▂▃▁▁▂▂
val accuracy,▁▄▅▆▆▆▇▇▇▇▇████████████████████


In [ ]:
criterion = nn.CrossEntropyLoss()
train_sparse_recursive(sparse_model, train_loader, val_loader, test_loader, criterion, hyperparams, device)

100%|██████████| 51/51 [00:00<00:00, 110.95it/s]


Epoch 1/64, Train Loss: 0.7693, Val Loss: 0.7440, Val Accuracy: 0.3832


100%|██████████| 51/51 [00:00<00:00, 125.68it/s]


Epoch 2/64, Train Loss: 0.7265, Val Loss: 0.7051, Val Accuracy: 0.4798


100%|██████████| 51/51 [00:00<00:00, 103.97it/s]


...

Epoch 62/64, Train Loss: 0.3828, Val Loss: 0.3899, Val Accuracy: 0.8336


100%|██████████| 51/51 [00:01<00:00, 41.43it/s]


Epoch 63/64, Train Loss: 0.3799, Val Loss: 0.3873, Val Accuracy: 0.8334
torch.Size([315]) torch.Size([3605])
combined_metrics torch.Size([3920])
mask torch.Size([3920])
tensor(1866)
num_emb_edges 315
tensor(121) tensor(470)
Chosen edges to del emb: tensor([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
          6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,
          7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,
          8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 14,
         14, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18,
         18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19],
        [11, 35, 67, 75, 78, 79, 83, 84, 85, 86, 87, 88, 89, 11, 35, 74, 75, 78,
         79, 83, 85, 86, 87, 89, 74, 75, 35, 78, 79, 83, 84, 85, 86, 87, 88, 89,
         11, 35, 67, 74, 75, 78, 79, 83, 

100%|██████████| 51/51 [00:01<00:00, 42.47it/s]


Epoch 64/64, Train Loss: 0.3791, Val Loss: 0.3867, Val Accuracy: 0.8339
